In [40]:
# Import necessary libraries
#from splinter import Browser
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from unidecode import unidecode
from colorama import Fore, init
import re
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np
import random

In [41]:
init() #Colorama init

random_wait = random.randint(2, 4)

#Red flags según artículo
flags_auto = ("conversable", "al dia", "detalles", "atrasada", "atrasado", "atrasados", "trizado", "trizada", "choques", "choque", "chocado", "deshuase", "desarme", "le falta", "roto", "rota", "dañado", "dañada", "malo", "mala", "repuesto", "repuestos", "no transferible", "multa", "multas", "parte", "partes")

flags = flags_auto

close_locations = ["Valparaiso", "Viña del Mar"]

In [91]:
login = False #True da más detalles de los autos pero puede ser problemático, False da menos detalles pero es más robusto

# Set up search parameters
radius = 250 # TIENE que ser [1, 2, 5, 10, 20, 40, 60, 80, 100, 250, 500]
search_location = "Valparaiso"
default_location = 111068758922766 #Curacaví, esto permite englobar stgo y valpo
min_price = 500000
max_price = 10000000
days_since_listed = 30
availability = f"in%20stock" 
# availability = f"out%20of%20stock" #para revisar lo que ya se vendió
min_km = 1000 #Menos de 1000 km suelen ser autos que omiten ceros
max_km = 300000
min_year = 1990
max_year = 2024
transmission = "manual"
make = "bmw "
search_model = ""
exact = "true" #false busca sugerencias, true busca exactamente lo que se pide

In [71]:
# Configurar el servicio de Chrome
service = Service(ChromeDriverManager().install())

# Configurar las opciones de Chrome
options = Options()
# options.add_argument("--headless")  # Para ejecutar Chrome en modo sin cabeza (sin ventana)
options.add_experimental_option("detach", True) # Para que el driver no se cierre al terminar el script
# deshabilitar notificaciones
options.add_argument("--disable-notifications")

# Inicializar el driver de Chrome
driver = webdriver.Chrome(service=service, options=options)
# Set up base url
base_url = "https://www.facebook.com/marketplace/"

#Set up full url
url = f"{base_url}{default_location}/search?availability={availability}&minPrice={min_price}&maxPrice={max_price}&daysSinceListed={days_since_listed}&maxMileage={max_km}&maxYear={max_year}&minMileage={min_km}&minYear={min_year}&transmissionType={transmission}&query={make}{search_model}&exact={exact}"

In [92]:
# Inicio de sesión o no

# Cuenta fake
mail = "roadtonostromo1@gmail.com"
password = "suufreejvrrr"

if login:

    driver.get(base_url)

    mail_input = driver.find_element(By.XPATH ,'/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[2]/form/div/div[3]/div/label/div/div/input')

    ActionChains(driver).click(mail_input).perform()

    mail_input.send_keys(mail)

    password_input = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[2]/form/div/div[4]/div/label/div/div/input')

    ActionChains(driver).click(password_input).perform()

    password_input.send_keys(password)

    mail_input.send_keys(Keys.RETURN)

    # Visitar el sitio web
    driver.get(url)

    # Esperar un tiempo breve para que la página se cargue completamente
    time.sleep(random_wait) # 1 segundo

else:
    # Visitar el sitio web
    driver.get(url)

    # Esperar un tiempo breve para que la página se cargue completamente
    time.sleep(random_wait) # 1 segundo

    # Buscar el elemento de la X para ignorar el inicio de sesión
    x_button = driver.find_element(By.XPATH, "//div[@aria-label='Cerrar']")

    # Hacer clic en la X para ignorar el inicio de sesión
    x_button.click()

Obtener información de la zona de búsqueda

In [45]:
change_location_selector = driver.find_element(By.XPATH, '//*[@id="seo_filters"]/div[1]')

# Obtener ubicación actual
current_location = driver.find_element(By.XPATH, '//*[@id="seo_filters"]/div[1]/div[1]/div/span').text
print(f"La búsqueda se está realizando en: {current_location}")

# Hacer clic en el selector de ubicación
# change_location_selector.click()

# Cambiar la ubicación de búsqueda
# location = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[2]/div/div/div/div/div/label/div/div[2]/input')

# ActionChains(driver).click(location).perform()

# esperar 1 segundo
time.sleep(random_wait)

La búsqueda se está realizando en: Curacaví, Region Metropolitana, Chile
  · En un radio de 65 kilómetros


In [46]:
#location.clear()

In [47]:
#location.send_keys(search_location)

#location.send_keys(Keys.RETURN)

# Hay que hacer que seleccione una de las opciones del menu desplegable

Facebook cambia los id de algunos elementos, cambiar el radio de busqueda como ultima alternativa ya que se debe volver a buscar el XPATH cada cierto tiempo

In [48]:
""" # buscar selector de radio
# radius_selector = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[3]/div/div/label')
radius_selector = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[4]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[3]/div/div/label/div/div[1]/div/div')                                               
# Hacer clic en el selector de radio
radius_selector.click()

# Botones de seleccion de radio

elementos = driver.find_elements(By.XPATH, "//div[@class='x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n xe8uvvx x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x87ps6o x1lku1pv x1a2a7pz x6s0dn4 xjyslct x9f619 x1ypdohk x78zum5 x1q0g3np x2lah0s xnqzcj9 x1gh759c xdj266r xat24cr x1344otq x1de53dj xz9dl7a xsag5q8 x1n2onr6 x16tdsg8 x1ja2u2z']")

# Selecciona el radio en el menú

for elemento in elementos:
    numero = re.match(r"\d+", elemento.text).group()
    
    if int(numero) == radius:
        elemento.click()
        break

# Buscar el botón de aplicar
apply_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[4]/div/div[2]/div/div/div/div/div')

# Hacer clic en el botón de aplicar
apply_button.click() """

' # buscar selector de radio\n# radius_selector = driver.find_element(By.XPATH, \'/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[3]/div/div/label\')\nradius_selector = driver.find_element(By.XPATH, \'/html/body/div[1]/div/div[1]/div/div[4]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[3]/div/div/label/div/div[1]/div/div\')                                               \n# Hacer clic en el selector de radio\nradius_selector.click()\n\n# Botones de seleccion de radio\n\nelementos = driver.find_elements(By.XPATH, "//div[@class=\'x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n xe8uvvx x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x87ps6o x1lku1pv x1a2a7pz x6s0dn4 xjyslct x9f619 x1ypdohk x78zum5 x1q0g3np x2lah0s xnqzcj9 x1gh759c xdj266r xat24cr x1344otq x1de53dj xz9dl7a xsag5q8 x1n2onr6 x16tdsg8 x1ja2u2z\']")\n\n# Selecciona el radio en el menú\n\nfor elemento in elementos:\n    num

In [93]:
# Número máximo de intentos para detectar si se ha alcanzado el final de la página
max_attempts = 100
# Tiempo máximo para esperar que ocurran cambios en la posición vertical de la barra de desplazamiento
max_scroll_wait = 3  # segundos

# Función para verificar si se ha alcanzado el final de la página
def is_page_end_reached(driver):
    current_scroll_position = driver.execute_script("return window.scrollY;")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)  # Esperar un momento para que se actualice la posición de desplazamiento
    new_scroll_position = driver.execute_script("return window.scrollY;")
    return new_scroll_position == current_scroll_position

# Desplazarse hacia abajo hasta que se haya alcanzado el final de la página o se haya intentado max_attempts veces
attempt = 1
while attempt <= max_attempts:
    if is_page_end_reached(driver):
        break
    attempt += 1
    time.sleep(4)  # Retraso antes de intentar nuevamente

# Si después de los intentos la página todavía no se ha cargado completamente, espera un tiempo adicional
if attempt > max_attempts:
    WebDriverWait(driver, max_scroll_wait).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )

# Ahora la página debe estar completamente cargada


In [94]:
# Obtener el HTML de la página
html = driver.page_source

# Crear un objeto BeautifulSoup a partir del HTML
market_soup = soup(html, 'html.parser')

In [51]:
""" # Escribir el HTML en un archivo
with open("hyundai.html", "w", encoding="utf-8") as file:
    file.write(html)

# Crear un objeto BeautifulSoup a partir del HTML
market_soup = soup(html, 'html.parser')
 """

' # Escribir el HTML en un archivo\nwith open("hyundai.html", "w", encoding="utf-8") as file:\n    file.write(html)\n\n# Crear un objeto BeautifulSoup a partir del HTML\nmarket_soup = soup(html, \'html.parser\')\n '

In [52]:
# Leer html desde archivo

""" with open("hyundai.html", "r", encoding="utf-8") as file:
    html = file.read()

# Crear un objeto BeautifulSoup a partir del HTML

market_soup = soup(html, 'html.parser') """

' with open("hyundai.html", "r", encoding="utf-8") as file:\n    html = file.read()\n\n# Crear un objeto BeautifulSoup a partir del HTML\n\nmarket_soup = soup(html, \'html.parser\') '

In [95]:
# Extract all the necessary info and insert into lists
titles_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
titles_list = [title.text.strip() for title in titles_div]
prices_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x676frb x1lkfr7t x1lbecb7 x1s688f xzsf02u")
prices_list = [price.text.strip() for price in prices_div]
locations_and_mileages_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x3x7a5m x1nxh6w3 x1sibtaa xo1l8bm xi81zsa")
locations_and_mileages_list = [location_and_mileage.text.strip() for location_and_mileage in locations_and_mileages_div]
urls_div = market_soup.find_all('a', class_="x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1lku1pv")
base_url = "https://web.facebook.com"
urls_list = [base_url+url.get('href') for url in urls_div]

In [54]:
# locations_and_mileages_list

# urls_list


In [97]:
# Create a regular expression pattern to match city and state entries like "City, State"
pattern = re.compile(r'(\w+(?:-\w+)?, [A-Z]{2})')

# Define regular expressions to extract numeric mileage values in "K km" and "K miles" format
mileage_pattern_km = r'(\d+) km'
mileage_pattern_miles = r'(\d+) millas'

# Initialize an empty list to store adjusted mileage entries
locations_and_mileages_list2 = []

# Iterate through the original mileage entries
for item in locations_and_mileages_list:
    # Append the current mileage entry to the adjusted list
    locations_and_mileages_list2.append(item)	

    # Check if the current mileage entry matches the pattern and there are at least two entries in the adjusted list
    if item == '':
        locations_and_mileages_list2[-1] = item.replace('', '0')
        print(f"Se encontró un espacio en blanco en el elemento {len(locations_and_mileages_list2)} y se rellenó con 0 km")

    elif pattern.search(item) and len(locations_and_mileages_list2) >= 2 and pattern.search(locations_and_mileages_list2[-2]):
        # If the conditions are met, insert "0K km" in between the two consecutive city and state entries
        locations_and_mileages_list2.insert(-1, '0')
        print(f"Se encontró un espacio en blanco en el elemento {len(locations_and_mileages_list2)} y se rellenó con 0 km")  

    
    # Try to find a match for the "K km" format
    match_mileage_km = re.search(mileage_pattern_km, locations_and_mileages_list2[-1])
        
    # Try to find a match for the "K miles" format
    match_mileage_miles = re.search(mileage_pattern_miles, locations_and_mileages_list2[-1])

    if "\xa0mil" in item: #Por esta parte se necesita que se busquen autos con mas de 1000 km
        locations_and_mileages_list2[-1] = item.replace("\xa0mil", "000")
        # Try to find a match for the "K km" format
        match_mileage_km = re.search(mileage_pattern_km, locations_and_mileages_list2[-1])
        
        # Try to find a match for the "K miles" format
        match_mileage_miles = re.search(mileage_pattern_miles, locations_and_mileages_list2[-1])
         
        
    # Check if either of the formats is found
    if match_mileage_km or match_mileage_miles:
        # If "K km" format is found, convert it to meters and append to the cleaned list
        if match_mileage_km:
            locations_and_mileages_list2[-1] = match_mileage_km.group(1)
        # If "K miles" format is found, convert it to meters and append to the cleaned list
        else:
            locations_and_mileages_list2[-1] = str((int(match_mileage_miles.group(1)) * 1609//1000))
            print(f"Se encontró una distancia en K millas {int(match_mileage_miles.group(1))}, se convirtió a {locations_and_mileages_list2[-1]} metros")

Se encontró una distancia en K millas 218000, se convirtió a 350762 metros


In [98]:
# Separate locations and mileages lists

# Initialize an empty list to store cleaned values
mileage_clean = []
locations_list = []

# Iterate through the locations_and_mileages_list2 in steps of 2 to separate locations and mileage
for i in range(0, len(locations_and_mileages_list2), 2):
    # Extract location
    location_parts = locations_and_mileages_list2[i].split(',')
    if len(location_parts) > 0:
        # Clean and decode the location to handle UTF-8 characters properly
        location = location_parts[0].strip().encode('utf-8').decode('utf-8')
        locations_list.append(location)   

    # Extract mileage
    mileage_parts = locations_and_mileages_list2[i + 1].split(' ')
    if len(mileage_parts) > 0:
        mileage = mileage_parts[0]
        try:
            mileage_clean.append(int(mileage))
        except:
            print(Fore.RED + locations_and_mileages_list2[i + 1] + Fore.RESET)

print("Locations:", locations_list)
print("Mileage:", mileage_clean)

Locations: ['Santiago', 'María Pinto', 'Santiago', 'Colina', 'Santiago', 'Quilpué', 'Ñuñoa', 'La Pintana', 'Ñuñoa', 'Puente Alto', 'Lampa', 'Independencia', 'Talagante', 'Renca', 'Peñaflor', 'Buin', 'Santiago', 'Rancagua', 'Quilpué', 'Viña del Mar', 'San Miguel', 'Cerrillos', 'Rancagua', 'Cerrillos', 'Pudahuel', 'Algarrobo', 'Talagante', 'Lo Prado', 'Talagante', 'Buin', 'Lampa', 'Santiago', 'Quilpué', 'Macul', 'Las Condes', 'Lo Barnechea', 'Viña del Mar', 'San Miguel', 'Conchalí', 'Lo Espejo', 'Talagante', 'Peñaflor', 'Vitacura', 'Las Condes', 'Pudahuel', 'Pudahuel', 'Rancagua', 'Santiago', 'Santiago', 'Cerrillos', 'La Serena', 'Santiago', 'Molina', 'Chillán Viejo', 'Santa Cruz', 'Vitacura', 'Puente Alto', 'Santiago', 'Puente Alto', 'Parral', 'Maipú', 'Santiago', 'Pichilemu', 'Los Andes', 'Santa Cruz', 'Santiago', 'Cerro Navia', 'La Serena', 'Santiago', 'Las Condes', 'San Javier', 'Peñalolén', 'Ovalle', 'La Florida', 'La Serena', 'Chépica', 'Santiago', 'Calera', 'San Ramón', 'Palmilla'

In [99]:
prices_list

['$3.900.000',
 '$9.890.000',
 '$4.000.000',
 '$8.500.000',
 '$8.990.000',
 '$6.100.000',
 '$4.000.000',
 '$3.300.000',
 '$4.000.000',
 '$9.500.000',
 '$6.500.000',
 '$777.777',
 '$6.900.000',
 '$5.800.000',
 '$9.000.000',
 '$6.750.000',
 '$2.700.000',
 '$4.350.000',
 '$8.400.000',
 '$7.000.000',
 '$4.690.000',
 '$4.690.000',
 '$7.990.000',
 '$8.580.000',
 '$4.700.000',
 '$6.999.000',
 '$6.750.000',
 '$3.500.000',
 '$6.900.000',
 '$7.700.000',
 '$5.450.000',
 '$7.000.000',
 '$4.950.000',
 '$7.500.000',
 '$1.000.000',
 '$5.000.000',
 '$5.500.000',
 '$6.800.000',
 '$3.900.000',
 '$6.500.000',
 '$9.999.999',
 '$3.800.000',
 '$9.280.000',
 '$9.500.000',
 '$4.700.000',
 '$3.250.000',
 '$3.000.000',
 '$7.000.000',
 '$8.990.000',
 '$5.100.000',
 '$5.600.000',
 '$5.100.000',
 '$5.290.000',
 '$3.100.000',
 '$5.800.000',
 '$8.990.000',
 '$8.990.000',
 '$9.800.000',
 '$7.600.000',
 '$6.599.990',
 '$8.400.000',
 '$6.950.000',
 '$8.400.000',
 '$8.500.000',
 '$6.000.000',
 '$7.380.000',
 '$5.200.000

In [101]:
# Ajustar formato según si está en stock o no
if availability == "out%20of%20stock":
    prices_list =  [elemento for elemento in prices_list if elemento != '·']
#prices_list
    
clean_prices_list = []
# Limpiar y convertir los precios
for precio in prices_list:
    # Remover espacios en blanco y símbolo "$", y convertir a entero
    try: 
        precio_limpio = int(precio.replace('\xa0', '').replace('$', ''))
        # Agregar el precio limpio a la lista
        clean_prices_list.append(precio_limpio)
        
    except:
        try: 
            precio_limpio = int(precio.replace('.', '').replace('$', ''))
            # Agregar el precio limpio a la lista
            clean_prices_list.append(precio_limpio)
        except:
            #Delete element from the lists
            index = prices_list.index(precio)
            print(Fore.RED + f"Se encontró un precio no válido: {precio}, en la posición {index}" + Fore.RESET)
            print(urls_list[index])
            del prices_list[index]
            del titles_list[index]
            del locations_list[index]
            del mileage_clean[index]
            del urls_list[index]
        
        

In [102]:
clean_prices_list

[3900000,
 9890000,
 4000000,
 8500000,
 8990000,
 6100000,
 4000000,
 3300000,
 4000000,
 9500000,
 6500000,
 777777,
 6900000,
 5800000,
 9000000,
 6750000,
 2700000,
 4350000,
 8400000,
 7000000,
 4690000,
 4690000,
 7990000,
 8580000,
 4700000,
 6999000,
 6750000,
 3500000,
 6900000,
 7700000,
 5450000,
 7000000,
 4950000,
 7500000,
 1000000,
 5000000,
 5500000,
 6800000,
 3900000,
 6500000,
 9999999,
 3800000,
 9280000,
 9500000,
 4700000,
 3250000,
 3000000,
 7000000,
 8990000,
 5100000,
 5600000,
 5100000,
 5290000,
 3100000,
 5800000,
 8990000,
 8990000,
 9800000,
 7600000,
 6599990,
 8400000,
 6950000,
 8400000,
 8500000,
 6000000,
 7380000,
 5200000,
 6500000,
 6500000,
 7500000,
 6300000,
 5800000,
 3590000,
 6500000,
 8700000,
 8650000,
 4900000,
 9500000,
 10000000,
 5800000,
 9000000,
 7600000,
 8500000,
 9980000,
 7500000,
 6900000,
 9000000,
 7950000,
 8500000,
 9000000,
 7000000,
 8200000,
 7850000,
 8500000,
 9050000,
 8550000,
 7900000,
 10000000,
 9000000,
 8000000,

In [103]:
def get_description(url):
    driver.get(url)
    
    
    try:
        # Buscar el elemento de la X para ignorar el inicio de sesión
        x_button = driver.find_element(By.XPATH, "//div[@aria-label='Cerrar']")
        # Hacer clic en la X para ignorar el inicio de sesión
        x_button.click()
        # Esperar un tiempo breve para que la página se cargue completamente
        # time.sleep(random_wait)
    except:
        pass

    try:

        ver_mas_button = driver.find_element(By.XPATH, "//div[contains(@class, 'x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x1n2onr6 x87ps6o x1lku1pv x1a2a7pz')]//*[contains(text(), 'Ver más')]")
        # Hacer clic en el botón "Ver más"
        ver_mas_button.click()
        # Esperar un tiempo breve para que la página se cargue completamente
        

    except:
        pass
    
    html = driver.page_source
    item_soup = soup(html, 'html.parser')

    #description_spans = item_soup.find('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x3x7a5m x6prxxf xvq8zen xo1l8bm xzsf02u")
    description_spans = item_soup.find('div', class_="xz9dl7a x4uap5 xsag5q8 xkhd6sd x126k92a")

    #Si no se encuentra la descripción, se le asigna "No description"
    if description_spans == None:
        description = "No description"
    else:
        description = description_spans.text.strip()

    words = description.split()  # Dividir la descripción en palabras
    if len(words) >= 2 and words[-2:] == ["Ver", "menos"]:
        # Si las últimas dos palabras son "Ver menos", eliminarlas
        description = " ".join(words[:-2]).strip()

    #print(description)
    time.sleep(random_wait)
    return description


La parte de filtrado de resultados obtenidos es la más demorada de todas

In [104]:
# Filter and add all values to a list of dictionaries
items_list = []

patron = re.compile(r'\b{}\b'.format(re.escape(make)), re.IGNORECASE)

for i, item in enumerate(titles_list):
    items_dict = {}
    
    title_split = titles_list[i].split()

    # Check if the title contains the search make
    if re.search(patron, item):

        items_dict["Titel"] = item
        items_dict["Year"] = int(title_split[0])
        items_dict["Make"] = title_split[1]
        #items_dict["Model"] = title_split[2]
        items_dict["Model"] = ' '.join(title_split[2:])
        items_dict["Price"] = clean_prices_list[i]
        items_dict["Search Model"] = search_model
        items_dict["Transmission"] = transmission
        items_dict["Mileage"] = mileage_clean[i]
        items_dict["Location"] = locations_list[i]
        items_dict["Description"] = get_description(urls_list[i]) #La parte más lenta

        detected_flags = []            

        # Check for flags in the description
        description = unidecode(items_dict["Description"].lower())  # Convertir la descripción a minúsculas para hacer la búsqueda sin distinción de mayúsculas y minúsculas y CREO que para quitar acentos
        for flag in flags:
            if re.search(r'\b{}\b'.format(re.escape(flag)), description):
                if flag == 'detalles' and ('detalles esteticos' in description or 'detalles de pintura' in description or 'paqueños detalles' in description or 'detalles menores' in description or 'detalles minimos' in description):
                    # Si encuentra detalles menores se guarda en los flags como "detalles menores"
                    detected_flags.append('detalles menores')

                if flag == 'multa' and ('ninguna multa' in description or 'sin multa' in description):
                    continue
                elif flag == 'multas' and ('sin multas' in description or 'no tiene multas' in description or 'libre de multas' in description or 'ni multas' in description or 'no multas' in description):
                    continue
                elif flag == 'parte' and ('ningun parte' in description):
                    continue
                elif flag == 'partes' and ('sin partes' in description or 'no tiene partes' in description or 'libre de partes' in description or 'ni partes' in description or 'no partes' in description):
                    continue
                elif flag == 'detalles' and ('sin detalles' in description or 'no tiene detalles'):
                    continue
                elif flag == 'detalle' and ('sin detalle' in description or 'ningun detalle' in description):
                    continue
                elif flag == 'choques' and ('sin choques' in description):
                    continue
                elif flag == 'chocado' and ('nunca chocado' in description):
                    continue
                else:
                    detected_flags.append(flag)

        if detected_flags:
            print(f"[{i}/{len(titles_list)}] Se encontraron los flags {detected_flags} en la siguiente descripción: {items_dict['Description']}")


        items_dict["Flags"] = detected_flags
        items_dict["URL"] = urls_list[i]

        items_list.append(items_dict)
    
#print(items_list)
print("\nSe obtuvieron "+ str(len(items_list)) + " resultados utilizables")
print("De una búsqueda de "+ str(len(titles_list)) +" resultados")


[0/285] Se encontraron los flags ['detalles menores'] en la siguiente descripción: BMW E46 del año 2002 con un motor de 2.0 litros y 235,000 kilómetros recorridos. Este vehículo tiene características destacadas: Frenos nuevos: Pastillas de freno cerámicas y discos nuevos Zimmerman. Mantenimiento reciente: Caja y motor revisados hace 15,000 km y 2,000 km respectivamente. Transmisión automática: 5 velocidades y 3 modos :automatico/sport/manual. Seguridad avanzada: Control de tracción, ABS y airbags. Comodidad interior: Aire acondicionado, sunroof, retrovisores eléctricos y cámara de retroceso. Entretenimiento moderno: Pantalla plug and play con USB, Bluetooth y controles en el volante. Funcionalidad adicional: Control de crucero, cierre centralizado y alarma. Estética deportiva: Llantas asimétricas aro 17 del modelo “look M”. Tiene detalles estéticos en el exterior que arreglar, tengo el parachoque trasero, pero hay que pintarlo para igualar al color de la carrocería, para mas detalles a

In [ ]:
# Cerrar la sesión del navegador
#driver.quit()

In [105]:
items_df = pd.DataFrame(items_list)

# Set the display option to ensure that all characters in a column are shown
pd.set_option('display.max_colwidth', None)

items_df.tail()

,Titel,Year,Make,Model,Price,Search Model,Transmission,Mileage,Location,Description,Flags,URL
132,2011 BMW x1,2011,BMW,x1,9100000,,manual,137000,Concepción,BMW X1 2011 Motor 2.0 bencinero Automático 137.xxx km Se vende o permuta (recibo o doy a favor) Valor conversable,[conversable],https://web.facebook.com/marketplace/item/315334391202055/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
133,2009 BMW series 5,2009,BMW,series 5,8000000,,manual,205000,Chillán,Se vende BMW 525i top line !! - transferible al día -Año: 2009 -Full cuero -Velocidad crucero -Asientos eléctricos -Espejos eléctricos -Motor 2.5 v6 en línea -Sunroff eléctrico -Computador abordo -205.000 kilómetros Ig: EM_ventas_automotora Consultas al DM,[al dia],https://web.facebook.com/marketplace/item/713214697504640/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
134,2011 BMW series 3,2011,BMW,series 3,9000000,,manual,158000,San Pedro de La Paz,Auto en excelente estado....se puede permutar por auto de menor valor mas diferencia...coordinar visitas,[],https://web.facebook.com/marketplace/item/1320558861862995/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
135,2006 BMW 525 ia,2006,BMW,525 ia,8000000,,manual,250000,Concepción,"Se vende vehículo en perfecto estado ,con sus manteciones ,tope de línea , vel cruzero,asientos de cuero,A/C,sensores de lluvia,sensor de retroceso delantero y trasero ,sunroof,espejos eléctricos,entre otros ,todo original ,se va con sus 2 llaves ,documentos al día [hidden information]",[al dia],https://web.facebook.com/marketplace/item/380127631476859/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
136,2008 BMW x5,2008,BMW,x5,10000000,,manual,205000,Los Ángeles,"Vehiculo en excelentes condiciones mecanicas, todos sus fluidos fueron cambiados hace 10.000km tanto de transmisión y motor con sus respectivos filtros. Papeles al dia y transferencia inmediata, detalles esteticos nada grave. Podria permutar o recibir en parte de pago. Recibo ofertas serias Se encuentra en los angeles.","[al dia, detalles menores, parte]",https://web.facebook.com/marketplace/item/756611033104304/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD


In [106]:
items_df

,Titel,Year,Make,Model,Price,Search Model,Transmission,Mileage,Location,Description,Flags,URL
0,2002 BMW e46 2.0 318i post facelift,2002,BMW,e46 2.0 318i post facelift,3900000,,manual,235000,Santiago,"BMW E46 del año 2002 con un motor de 2.0 litros y 235,000 kilómetros recorridos. Este vehículo tiene características destacadas: Frenos nuevos: Pastillas de freno cerámicas y discos nuevos Zimmerman. Mantenimiento reciente: Caja y motor revisados hace 15,000 km y 2,000 km respectivamente. Transmisión automática: 5 velocidades y 3 modos :automatico/sport/manual. Seguridad avanzada: Control de tracción, ABS y airbags. Comodidad interior: Aire acondicionado, sunroof, retrovisores eléctricos y cámara de retroceso. Entretenimiento moderno: Pantalla plug and play con USB, Bluetooth y controles en el volante. Funcionalidad adicional: Control de crucero, cierre centralizado y alarma. Estética deportiva: Llantas asimétricas aro 17 del modelo “look M”. Tiene detalles estéticos en el exterior que arreglar, tengo el parachoque trasero, pero hay que pintarlo para igualar al color de la carrocería, para mas detalles al interno Lo muestro en la florida Si esta publicado, esta disponible",[detalles menores],https://web.facebook.com/marketplace/item/818899626922095/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
1,2010 BMW series 5,2010,BMW,series 5,9890000,,manual,125000,María Pinto,SE VENDE O SE PERMUTA BMW 523i AÑO 2010 FULL CUERO MOTOR 2.5 CAJA AT CLIMATIZADOR VELOCIDAD CRUCERO ESCOTILLA MALETERO ELÉCTRICO VIDRIOS POLARIZADOS 120.XXX KM REALES ENTRE MÁS COSAS VEHÍCULO AL DÍA TRASFERENCIA INMEDIATA PRECIO FIJO SE RECIBE VEHÍCULO EN PARTE DE PAGO PERMUTA 11M MÁS INFO AL [hidden information],"[al dia, parte]",https://web.facebook.com/marketplace/item/816169343672280/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
2,2008 BMW series 3,2008,BMW,series 3,4000000,,manual,1000,Santiago,"Lo compre con detalles de caja nunca le hicieron cambio de aceite a la caja costaba mucho que entraran los cambios. Yo lo recupere le pusimos aditivo para recuperar un poco el aceite logrando que entraran todos los cambios, tienen que andarlo mínimos 150 kilómetros con el aditivo y hacerle el cambio de aceite a la caja conjunto con el filtro y queda listo el problema que tenía ya entran los cambios y todo pero no lo ocupo ya que por falta de espacio y tiempo no e podido darle el rodaje. Ideal cambiarle bomba de agua por seguridad yo lo ocupo de ves en cuando y no tiene problemas pero tiene su consumo de agua dependiendo la distancia solo por seguridad. Esta revisión técnica al día falta seguro y permiso tiene multas y se supone que está en prenda pero hablando con el dueño anterior me dejo transferencia abierta y todo el tema. Prenda antigua. Estuvo votado mucho tiempo tiene sus detalles pero el autito muy cómodo versión full aire acondicionado todo precio conversable.","[conversable, al dia, multas]",https://web.facebook.com/marketplace/item/4102988583261446/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
3,2014 BMW x1 x-line,2014,BMW,x1 x-line,8990000,,manual,117000,Santiago,BMW X1 versión X-line Motor 2.0 bencinero turbo Transmisión automática Sunrooof 116.xxx kilometraje Documentos al día transferible,[al dia],https://web.facebook.com/marketplace/item/1735482846978679/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
4,2009 BMW 318i trip tronic,2009,BMW,318i trip tronic,6100000,,manual,17000,Quilpué,SE VENDE ULTIMO MES AUTO EN BUENAS CONDICIONES CUALQUIER COSA INBOX PRECIO UNICO SE VENDE POR VIAJE,[],https://web.facebook.com/marketplace/item/348811508153383/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
...,...,...,...,...,...,...,...,...,...,...,...,...
132,2011 BMW x1,2011,BMW,x1,9100000,,manual,137000,Concepción,BMW X1 2011 Motor 2.0 bencinero Automático 137.xxx km Se vende o permuta (recibo o doy a favor) Valor conversable,[conversable],https://web.facebook.c

In [107]:
csv_name = f"{make.upper()}_{transmission.upper()}.csv"
# Guardar el DataFrame en un archivo CSV
items_df.to_csv(csv_name, index=False)  # El parámetro index=False evita que se agregue una columna de índice al archivo CSV

In [108]:
# Extraer los precios y URLs de la lista de diccionarios
prices = [item["Price"] for item in items_list]
urls = [item["URL"] for item in items_list]

# Calcular el precio promedio
precio_promedio = np.mean(prices)

# Calcular el precio más barato
precio_mas_barato = np.min(prices)
indice_precio_mas_barato = np.argmin(prices)

# Calcular el percentil 10 más bajo
percentil_10_mas_bajo = np.percentile(prices, 10)

# Calcular la mediana de los precios
mediana_precios = np.median(prices)


# Imprimir los resultados
print("Precio promedio:", precio_promedio)
print("Mediana de precios:", mediana_precios)
print("Percentil 10 más bajo:", percentil_10_mas_bajo)
print("Precio más barato:", precio_mas_barato)
print("URL:", urls[indice_precio_mas_barato])

Precio promedio: 7210844.262773722
Mediana de precios: 7990000.0
Percentil 10 más bajo: 3860000.0
Precio más barato: 777777
URL: https://web.facebook.com/marketplace/item/1126616618550907/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
